#### Colab prep

In [ ]:
!pip install hyperopt
!pip install guildai # restart after install

In [ ]:
# colab prep
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/colab-handover/autochord/* ./
!ls

<hr style="border:1px solid gray">

In [1]:
import pandas as pd
base_dir = 'data/McGill-Billboard'
data_index = 'billboard-2.0-manychords.csv'

df_songs = pd.read_csv(f'{base_dir}/{data_index}')
df_songs.set_index('id', inplace=True)
len(df_songs)

719

In [2]:
df_songs.head(n=3)

,title,artist,no_chord_percent
id,,,
3,I Don't Mind,James Brown,0.049747
4,You've Got A Friend,"Roberta Flack,Donny Hathaway",0.050770
6,The Rose,Bette Midler,0.117244


In [3]:
test_ids = [1289, 736, 637, 270, 18] # songs to exclude for testing
df_dataset = df_songs.drop(index=test_ids)
len(df_dataset)

714

#### Splitting

In [4]:
import numpy as np
_SEED = 0

df_idxs = np.array(df_dataset.index.values)
rng = np.random.default_rng(_SEED)
rng.shuffle(df_idxs)

df_idxs[:10]

array([1167,    6,  986,  227,  743,  568,  107,  181,   27,  793])

In [5]:
from dataloader import ChromaSequenceDataset
# import dataloader
# from importlib import reload
# reload(dataloader)

_LABEL_TYPE = 'majmin'
_SEQ_LEN = 64

pre_computed_seq = f'data/chordseq/{_LABEL_TYPE}_{_SEQ_LEN}.pkl'
ds = ChromaSequenceDataset(pre_computed_sequence=pre_computed_seq)

Loaded sequence data.


In [6]:
for train_split, val_split in ds.get_next_cv_split(df_idxs):
    print(train_split.shape, val_split.shape)

((43541, 64, 24), (43541, 64)) ((6882, 64, 24), (6882, 64))
((43575, 64, 24), (43575, 64)) ((6848, 64, 24), (6848, 64))
((43448, 64, 24), (43448, 64)) ((6975, 64, 24), (6975, 64))
((43485, 64, 24), (43485, 64)) ((6938, 64, 24), (6938, 64))
((42843, 64, 24), (42843, 64)) ((7580, 64, 24), (7580, 64))


### Training loop

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

def K_plot_loss(history):
    plt.plot(history.history['crf_loss'])
    plt.plot(history.history['val_crf_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [8]:
from dataloader import _CHROMA_FEAT_NAMES, _MAJMIN_CLASSES
# import model
# from importlib import reload
# reload(model)
from model import ModelWithCRFLoss
from tensorflow.keras.models import Model

_SEQ_LEN = 64

def init_bilstm_crf_model(base_linear_units=256, dropout=0.6, opt='adam', lr=0.01):
    input_ph = tf.keras.Input(shape=(_SEQ_LEN, len(_CHROMA_FEAT_NAMES),))
    lstm_out = layers.Bidirectional(
        layers.LSTM(units=base_linear_units, dropout=dropout,
                    return_sequences=True, stateful=False),
        merge_mode='concat')(input_ph)
    crf_out = tfa.layers.CRF(units=len(_MAJMIN_CLASSES))(lstm_out)
    model = Model(input_ph, crf_out)
    model = ModelWithCRFLoss(model, dtype='float64')
    
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, metrics=['accuracy'])
    
    return model

model = init_bilstm_crf_model()

In [9]:
model.base_model.summary()
model.base_model.output_shape

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 24)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64, 512)           575488    
_________________________________________________________________
crf (CRF)                    [(None, 64), (None, 64, 2 13500     
Total params: 588,988
Trainable params: 588,988
Non-trainable params: 0
_________________________________________________________________


[(None, 64), (None, 64, 25), (None,), (25, 25)]

In [10]:
_SEED = 0
_EPOCHS = 2 #10
_BATCH_SIZE = 2 #512
_CKPT_PATH = 'models/chroma-seq-bilstm-crf-{cv}'

def sample_train_loop(ds, ref_idxs):
    # cross validation loop
    tf.random.set_seed(_SEED)
    for cv_ix, (train, val) in enumerate(ds.get_next_cv_split(ref_idxs)):
        print(f'----------- CV{cv_ix+1} -----------')
        train_dataset = tf.data.Dataset.from_tensor_slices((train.feats, train.labels)) \
                                       .take(100) \
                                       .shuffle(buffer_size=len(train), seed=_SEED, reshuffle_each_iteration=True) \
                                       .batch(_BATCH_SIZE)
        val_dataset = tf.data.Dataset.from_tensor_slices((val.feats, val.labels)) \
                                     .take(100) \
                                     .shuffle(buffer_size=len(val), seed=_SEED, reshuffle_each_iteration=True) \
                                     .batch(_BATCH_SIZE)

        print(f'Num train: {len(train)}, Num val: {len(val)}')
        assert(train.feats.shape[1:] == val.feats.shape[1:])
        print(f'Input features: {train.feats.shape[1:]}, Num classes: {len(_MAJMIN_CLASSES)}')

        model = init_bilstm_crf_model()
        history = model.fit(train_dataset, validation_data=val_dataset, epochs=_EPOCHS)

        # get acc
        preds, _, _, _ = model.predict(val_split.feats, batch_size=_BATCH_SIZE)
        acc = accuracy_score(val_split.labels.flatten(), preds.flatten())
        print(f'Acc: {acc}')
        K_plot_loss(history)

        #model.save(_CKPT_PATH.format(cv=cv_ix))
        break

sample_train_loop(ds, df_idxs)

#### Automated tuning & tracking

In [ ]:
# colab
!cp -R /content/drive/MyDrive/colab-handover/autochord/guild-env-colab ./

In [11]:
from IPython.display import display

GUILD_HOME = 'guild-env/chroma-seq-bilstm-crf' # "guild-env-colab/simple-chroma"
DELETE_RUNS_ON_INIT = False
import guild.ipy as guild
guild.set_guild_home(GUILD_HOME)

if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))
else:
    display(guild.runs().head())

,run,operation,started,status,label


In [14]:
_EPOCHS = 1
_TRAIN = None
_VAL = None

# function for guild tracking
def hpset_trainloop(hd=256, dp=0.6, opt='adam', lr=0.001, bs=512, si=0):
    '''
    Train loop with a specific set of hyperparams
    
    hd: hidden dim base size
    dp: dropout rate
    opt: optimizer, lr: learning rate
    bs: batch size
    si: CV split index
    '''
    tf.random.set_seed(_SEED)
    train = _TRAIN
    val = _VAL
    if (not train) or (not val):
        raise Exception("Missing data!")
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train.feats, train.labels)) \
                                   .take(100) \
                                   .shuffle(buffer_size=len(train), seed=_SEED, reshuffle_each_iteration=True) \
                                   .batch(bs)
    val_dataset = tf.data.Dataset.from_tensor_slices((val.feats, val.labels)) \
                                 .take(100) \
                                 .shuffle(buffer_size=len(val), seed=_SEED, reshuffle_each_iteration=True) \
                                 .batch(bs)

    assert(train.feats.shape[-1] == val.feats.shape[-1])

    model = init_bilstm_crf_model(base_linear_units=hd, dropout=dp, opt=opt, lr=lr)
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=_EPOCHS, verbose=0)
    
    # get acc
    preds, _, _, _ = model.predict(val_split.feats, batch_size=_BATCH_SIZE)
    acc = accuracy_score(val_split.labels.flatten(), preds.flatten())
    
    best_epoch = np.argmin(history.history['val_crf_loss'])
    best_loss = history.history['val_crf_loss'][best_epoch]
    
    # output metrics
    print(f"BE: {best_epoch+1}")
    print(f"BL: {best_loss}")
    print(f"VA: {acc}")
    
    return acc

In [ ]:
# tuning loop
from hyperopt import hp, tpe, fmin

_REF_IDXS = df_idxs

def tuning_loop(hparams):
    global _TRAIN
    global _VAL
    global _REF_IDXS
    
    print(hparams)

    avg_acc = 0.0
    num_runs = 0
    for cv_ix, (train, val) in enumerate(ds.get_next_cv_split(_REF_IDXS)):
        _TRAIN = train
        _VAL = val
        run, acc = guild.run(hpset_trainloop,
                             hd=int(hparams['base_hidden_dim']),
                             dp=hparams['drop_rate'],
                             opt=hparams['opt'],
                             lr=hparams['lr'], 
                             bs=int(hparams['batch_size']),
                             si=cv_ix)
        
        num_runs += 1
        # if hyperparams fail miserably on one split,
        # no need to check other splits
        if acc < 0.5:
            return 1.0
            
        avg_acc += acc
    
    avg_acc /= num_runs
    return (1-avg_acc) # since we're using fmin

hparams = {
    'base_hidden_dim': hp.choice('base_hidden_dim', [256, 512, 1024]),
    'drop_rate': hp.choice('drop_rate', [0.9, 0.7, 0.5, 0.3]),
    'opt': hp.choice('opt', ['adam']),
    'lr': hp.choice('lr', [1e-2, 1e-3, 3e-4, 1e-4]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
}

best = fmin(tuning_loop, hparams, algo=tpe.suggest, max_evals=1)
print(best)

In [ ]:
# colab
!cp -R guild-env-colab /content/drive/MyDrive/colab-handover/autochord/

In [18]:
runs = guild.runs()
df_exps = runs.compare()

_COMPARE_COLS = ['bs','dp','hd','lr','opt','si','BE','BL','VA']
comps = df_exps[_COMPARE_COLS][:65]
comps[comps.VA > 0.52]

,bs,dp,hd,lr,opt,si,BE,VA
9,256,0.6,256,0.0003,adam,0,2.0,0.52030
12,512,0.3,256,0.0003,adam,2,2.0,0.52078
17,512,0.3,512,0.0003,adam,2,1.0,0.52138
36,256,0.5,256,0.0001,adam,3,2.0,0.52239
37,256,0.5,256,0.0001,adam,2,1.0,0.52428
39,256,0.5,256,0.0001,adam,0,2.0,0.52686
51,256,0.5,256,0.0003,adam,0,1.0,0.52050
53,256,0.5,256,0.0001,adam,0,2.0,0.52666


#### Scratch